In [38]:
from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN
from keras import initializers
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import gensim, logging
from gensim.models import word2vec
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from numpy.random import randint
from numpy import argmax
from keras.utils.np_utils import to_categorical

from sklearn.metrics import f1_score, precision_score, recall_score

In [39]:
batch_size = 32
num_classes = 41
epochs = 50
hidden_units = 100

In [40]:

learning_rate = 1e-6
clip_norm = 1.0

In [41]:
with open("filtered.txt","r") as readfile:
    types = [line.split("_") for line in readfile]

In [42]:
X=[]
y=[]
c=0
#print (types)
for i in range(len(types)):
    X.append(types[i][0])
    y.append(types[i][1])

label = [k.strip("\n") for k in y]
#print(label)
X = [X]

In [43]:
print(len(y))

52343


In [44]:
label = [k.strip("\n") for k in y]
#print(label)

In [45]:
model = gensim.models.Word2Vec(X, size=100, window=5, min_count=1, workers=4) #input shape=100

In [46]:
len(set(label))

13

In [47]:
m,n = np.shape(X)
dataset = []
for i in range(n):
    dataset.append(model.wv[X[0][i]])


In [48]:
le = preprocessing.LabelEncoder()

tags = le.fit(label)
le.fit(label)

tags=le.transform(label)

In [49]:
X_train, X_test, y_train, y_test = train_test_split(dataset, tags, test_size=0.20, random_state=42)

In [51]:
import numpy as np
print(np.shape(X_train), np.shape(X_test), np.shape(y_train), np.shape(y_test))


# In[30]:


num_classes = len(set(label))

(41874, 100) (10469, 100) (41874,) (10469,)


In [52]:
X_train = np.stack(X_train, axis = 0)
print(np.shape(X_train.shape[0]))
X_test = np.stack(X_test, axis = 0)
print(np.shape(X_test.shape[0]))
X_train = X_train.reshape(X_train.shape[0], -1, 1)
X_test = X_test.reshape(X_test.shape[0], -1, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

()
()
X_train shape: (41874, 100, 1)
41874 train samples
10469 test samples


In [53]:
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)
print(np.shape(X_train[0]))
print(np.shape(X_test[0]))
print(np.shape(y_test))
print(np.shape(X_test))

(100, 1)
(100, 1)
(10469, 13)
(10469, 100, 1)


In [54]:
model = Sequential()
model.add(SimpleRNN(hidden_units,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='tanh',
                    input_shape=X_train.shape[1:]))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
rmsprop = RMSprop(lr=learning_rate)
model.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [55]:
y_test = y_test.reshape(y_test.shape[0], -1, 1)

In [56]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
validation_data=(X_test, y_test))

Epoch 1/50
1309/1309 [==============================] - 33s 25ms/step - loss: 0.3886 - accuracy: 0.4640 - val_loss: 0.2645 - val_accuracy: 0.5426
Epoch 2/50
1309/1309 [==============================] - 32s 24ms/step - loss: 0.2337 - accuracy: 0.5449 - val_loss: 0.2124 - val_accuracy: 0.5426
Epoch 3/50
1309/1309 [==============================] - 32s 24ms/step - loss: 0.1996 - accuracy: 0.5449 - val_loss: 0.1905 - val_accuracy: 0.5426
Epoch 4/50
1309/1309 [==============================] - 32s 24ms/step - loss: 0.1835 - accuracy: 0.5449 - val_loss: 0.1787 - val_accuracy: 0.5426
Epoch 5/50
1309/1309 [==============================] - 32s 24ms/step - loss: 0.1740 - accuracy: 0.5449 - val_loss: 0.1712 - val_accuracy: 0.5426
Epoch 6/50
1309/1309 [==============================] - 32s 24ms/step - loss: 0.1677 - accuracy: 0.5449 - val_loss: 0.1659 - val_accuracy: 0.5426
Epoch 7/50
1309/1309 [==============================] - 32s 24ms/step - loss: 0.1632 - accuracy: 0.5449 - val_loss: 0.1621 -

In [57]:
score = model.evaluate(X_test, y_test, batch_size=batch_size)
print('Test score:', score)

328/328 [==============================] - 3s 8ms/step - loss: 0.1436 - accuracy: 0.5426
Test score: [0.14360906183719635, 0.5425541996955872]


In [58]:
y_true=y_test
y_pred=model.predict(X_test)

In [59]:
yp = []
for i in range(len(y_pred)):
    yp.append(np.argmax(y_pred[i]))

In [60]:
yt=y_test
yt=np.argmax(yt, axis=1)

In [61]:
yt = np.concatenate(yt).ravel().tolist()


In [62]:
f1_score(yt, yp, average='macro')  

0.06394992090700803